In [1]:
import pandas as pd
import Constants as c
away = 'Brooklyn Nets'
home = 'Miami Heat'
current_period = 4

In [2]:
#score_by_q = pd.read_csv('score_by_q_Los Angeles Lakers_Miami Heat.csv')
score_by_q = pd.read_csv('score_by_q_Brooklyn Nets_Miami Heat.csv')
score_by_q  = score_by_q [score_by_q ['player'].notna()].reset_index(drop=True)


In [3]:
# get a list of the players on each team
for player in pd.unique(score_by_q['player'].dropna()).tolist():
    try:
        team = pd.unique(score_by_q[score_by_q['player'] == player]['team'].dropna())[0]
    except IndexError:
        team = ''
    score_by_q.loc[(score_by_q['player'] == player) & (score_by_q['team'].isna()), 'team'] = team

In [4]:
score_by_q

,time_stamp,time_index,same_time_count,period,away,home,total,play,player,fg_makes,fg_misses,3_makes,3_misses,ft_makes,ft_misses,points,team
0,4 0.0 107 109 1,4 0.0 107 109,1,4,107,109,216,Bam Adebayo makes 15-foot pullup jump shot,Bam Adebayo,1.0,NaN,NaN,NaN,NaN,NaN,2.0,Miami Heat
1,4 24.3 107 107 1,4 24.3 107 107,1,4,107,107,214,Kyrie Irving misses shot,Kyrie Irving,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Brooklyn Nets
2,4 43.6 107 107 2,4 43.6 107 107,2,4,107,107,214,Goran Dragic makes two point shot,Goran Dragic,1.0,NaN,NaN,NaN,NaN,NaN,2.0,Miami Heat
3,4 48.9 107 105 1,4 48.9 107 105,1,4,107,105,212,Kyrie Irving misses 26-foot step back jumpshot,Kyrie Irving,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Brooklyn Nets
4,4 1:13 107 105 1,4 1:13 107 105,1,4,107,105,212,Duncan Robinson misses 26-foot three point jumper,Duncan Robinson,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Miami Heat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,1 10:37 6 5 1,1 10:37 6 5,1,1,6,5,11,Trevor Ariza misses 12-foot two point shot,Trevor Ariza,NaN,1.0,NaN,NaN,NaN,NaN,NaN,Miami Heat
203,1 10:52 6 5 1,1 10:52 6 5,1,1,6,5,11,Kevin Durant makes 25-foot three pointer,Kevin Durant,NaN,NaN,1.0,NaN,NaN,NaN,3.0,Brooklyn Nets
204,1 11:06 3 5 1,1 11:06 3 5,1,1,3,5,8,Bam Adebayo makes 6-foot two point shot (Kendr...,Bam Adebayo,1.0,NaN,NaN,NaN,NaN,NaN,2.0,Miami Heat
205,1 11:26 3 3 1,1 11:26 3 3,1,1,3,3,6,Kevin Durant makes 27-foot three point jumper ...,Kevin Durant,NaN,NaN,1.0,NaN,NaN,NaN,3.0,Brooklyn Nets


In [5]:
score_by_q.fillna(0, inplace=True)
# fills with one or zeros
one_hot = pd.get_dummies(score_by_q['period'])
one_hot = one_hot.T.reindex(list(range(1, current_period + 1))).T.fillna(0)

#points * 1 = amount of points the player has in the quarter
one_hot_points = one_hot[list(range(1, current_period + 1))].multiply(score_by_q['points'], axis='index')
one_hot_points = one_hot_points.rename(index=str, columns=c.period_points).reset_index(drop=True)

one_hot_fg_makes = one_hot[list(range(1, current_period + 1))].multiply(score_by_q['fg_makes'], axis='index')
one_hot_fg_makes = one_hot_fg_makes.rename(index=str, columns=c.period_fg_makes).reset_index(drop=True)
one_hot_fg_misses = one_hot[list(range(1, current_period + 1))].multiply(score_by_q['fg_misses'], axis='index')
one_hot_fg_misses = one_hot_fg_misses.rename(index=str, columns=c.period_fg_misses).reset_index(drop=True)

one_hot_ft_makes = one_hot[list(range(1, current_period + 1))].multiply(score_by_q['ft_makes'], axis='index')
one_hot_ft_makes = one_hot_ft_makes.rename(index=str, columns=c.period_ft_makes).reset_index(drop=True)
one_hot_ft_misses = one_hot[list(range(1, current_period + 1))].multiply(score_by_q['ft_misses'], axis='index')
one_hot_ft_misses = one_hot_ft_misses.rename(index=str, columns=c.period_ft_misses).reset_index(drop=True)

one_hot_3_makes = one_hot[list(range(1, current_period + 1))].multiply(score_by_q['3_makes'], axis='index')
one_hot_3_makes = one_hot_3_makes.rename(index=str, columns=c.period_3_makes).reset_index(drop=True)
one_hot_3_misses = one_hot[list(range(1, current_period + 1))].multiply(score_by_q['3_misses'], axis='index')
one_hot_3_misses = one_hot_3_misses.rename(index=str, columns=c.period_3_misses).reset_index(drop=True)

In [6]:
score_by_q_s = score_by_q[['player', 'team', 'points']]
new = pd.concat([score_by_q_s, one_hot_points, one_hot_fg_makes, one_hot_fg_misses, one_hot_3_makes, one_hot_3_misses, one_hot_ft_makes, one_hot_ft_misses], axis=1, ignore_index=False)
#print(score_by_q[score_by_q['player'] == 'LeBron James'])
grouped_score = new.groupby(['player', 'team'], as_index=False).sum()

fg_df = pd.DataFrame(columns=c.fg_cols)

for i in range(1, current_period+1):
    fg_attempts = grouped_score[str(i)+'Q FG'].astype(int) + grouped_score[str(i)+'Q FGM'].astype(int)
    fg_df[str(i)+'Q FG-FGA'] = grouped_score[str(i)+'Q FG'].astype(int).astype(str).str.cat(fg_attempts.astype(str), sep ="-")

    ft_attempts = grouped_score[str(i)+'Q FT'].astype(int) + grouped_score[str(i)+'Q FTM'].astype(int)
    fg_df[str(i)+'Q FT-FTA'] = grouped_score[str(i)+'Q FT'].astype(int).astype(str).str.cat(ft_attempts.astype(str), sep ="-")

    three_attempts = grouped_score[str(i)+'Q 3P'].astype(int) + grouped_score[str(i)+'Q 3PM'].astype(int)
    fg_df[str(i)+'Q 3P-3PA'] = grouped_score[str(i)+'Q 3P'].astype(int).astype(str).str.cat(three_attempts.astype(str), sep ="-")


In [7]:
final = pd.concat([grouped_score[['player', 'team', 'points']],fg_df], axis=1, ignore_index=False)
order = pd.CategoricalDtype([away, home], ordered=True)
final['team'] = final['team'].astype(order)
final.sort_values('team', inplace=True)
final

,player,team,points,1Q FG-FGA,1Q 3P-3PA,1Q FT-FTA,2Q FG-FGA,2Q 3P-3PA,2Q FT-FTA,3Q FG-FGA,3Q 3P-3PA,3Q FT-FTA,4Q FG-FGA,4Q 3P-3PA,4Q FT-FTA
0,Alize Johnson,Brooklyn Nets,3.0,0-0,1-1,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0
16,Timothe Luwawu-Cabarrot,Brooklyn Nets,7.0,0-0,1-1,0-0,1-1,0-1,0-0,1-1,0-0,0-0,0-1,0-0,0-0
15,Nicolas Claxton,Brooklyn Nets,4.0,1-1,0-0,0-0,0-1,0-0,0-0,1-1,0-0,0-0,0-0,0-0,0-0
14,Landry Shamet,Brooklyn Nets,30.0,1-1,2-3,0-0,2-2,2-4,0-0,0-0,1-3,1-2,0-0,2-2,2-2
13,Kyrie Irving,Brooklyn Nets,20.0,1-3,0-1,0-0,1-2,2-2,4-4,2-4,0-2,0-0,0-1,0-2,2-2
12,Kevin Durant,Brooklyn Nets,8.0,1-1,2-2,0-2,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0
9,Joe Harris,Brooklyn Nets,12.0,0-0,0-2,0-0,1-1,0-0,0-0,1-1,1-3,0-0,1-1,1-4,0-0
8,Jeff Green,Brooklyn Nets,8.0,1-1,0-0,0-0,1-2,0-0,0-0,0-1,0-2,0-0,1-2,0-1,2-2
4,Bruce Brown,Brooklyn Nets,8.0,0-0,0-1,0-0,0-1,0-0,0-0,2-2,0-0,0-2,2-3,0-0,0-0
3,Blake Griffin,Brooklyn Nets,7.0,0-1,1-1,0-0,0-0,0-1,1-2,0-2,1-1,0-0,0-0,0-1,0-0


In [8]:
#final['site'] = final['team'].apply(lambda x: 1 if x == home else (0 if x == away else x))
grouped_score['initials'] = grouped_score['player'].apply(lambda x: ' '.join([x.split()[0][0]]+ x.split()[1:]))
grouped_score['team'] = grouped_score['team'].map(c.format_team_names)
grouped_score

,player,team,points,1Q PTS,2Q PTS,3Q PTS,4Q PTS,1Q FG,2Q FG,3Q FG,...,4Q 3PM,1Q FT,2Q FT,3Q FT,4Q FT,1Q FTM,2Q FTM,3Q FTM,4Q FTM,initials
0,Alize Johnson,Nets,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A Johnson
1,Andre Iguodala,Heat,8.0,0.0,5.0,3.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,A Iguodala
2,Bam Adebayo,Heat,21.0,2.0,8.0,2.0,9.0,1.0,3.0,1.0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,B Adebayo
3,Blake Griffin,Nets,7.0,3.0,1.0,3.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,B Griffin
4,Bruce Brown,Nets,8.0,0.0,0.0,4.0,4.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,B Brown
5,Dewayne Dedmon,Heat,10.0,2.0,6.0,2.0,0.0,0.0,1.0,0.0,...,0.0,2.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,D Dedmon
6,Duncan Robinson,Heat,11.0,3.0,3.0,2.0,3.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D Robinson
7,Goran Dragic,Heat,18.0,8.0,3.0,3.0,4.0,0.0,0.0,1.0,...,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,G Dragic
8,Jeff Green,Nets,8.0,2.0,2.0,0.0,4.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,J Green
9,Joe Harris,Nets,12.0,0.0,2.0,5.0,5.0,0.0,1.0,1.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,J Harris


In [35]:
players = pd.Series(['B Adebayo', 'K Nunn','A Iguodala', 'T Herro', 'D Robinson', 'K Irving',  'L Shamet', 'B Brown', 'J Harris', 'J Green'])
teams = pd.Series(['Heat', 'Heat', 'Heat', 'Heat', 'Heat', 'Nets', 'Nets', 'Nets', 'Nets', 'Nets'])
otf = pd.DataFrame({'initials': players, 'team':teams})
#ts percentage for each team with the players they have on the floor
otf2 = otf.merge(grouped_score, how='left', on=['initials','team'], )
otf_df = otf2.groupby('team', as_index=False).sum()
otf_df

,team,points,1Q PTS,2Q PTS,3Q PTS,4Q PTS,1Q FG,2Q FG,3Q FG,4Q FG,...,3Q 3PM,4Q 3PM,1Q FT,2Q FT,3Q FT,4Q FT,1Q FTM,2Q FTM,3Q FTM,4Q FTM
0,Heat,66.0,13.0,21.0,15.0,17.0,2.0,5.0,4.0,5.0,...,5.0,2.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0
1,Nets,78.0,12.0,26.0,17.0,23.0,3.0,5.0,5.0,4.0,...,8.0,6.0,0.0,4.0,1.0,6.0,0.0,0.0,3.0,0.0


In [80]:
import numpy as np

def advanced_stats(teams_df):
    shots = pd.DataFrame(0, index=np.arange(len(teams_df)), columns=['attempts', '3p_attempts'])

    points, fg_attempts, ft_attempts, three_attempts = 0, 0, 0, 0

    for i in range(1, current_period+1):
        #points += teams_df[str(i)+'Q PTS'].astype(int)
        #print(i)
        fg_attempts += (teams_df[str(i)+'Q FG'].sum() + teams_df[str(i)+'Q FGM'].sum())
        ft_attempts += (teams_df[str(i)+'Q FT'].sum() + teams_df[str(i)+'Q FTM'].sum())
        three_attempts += (teams_df[str(i)+'Q 3P'].sum() + teams_df[str(i)+'Q 3PM'].sum())
        shots['attempts'] +=  teams_df[str(i)+'Q FG'] + teams_df[str(i)+'Q FGM'] +teams_df[str(i)+'Q 3P']+ teams_df[str(i)+'Q 3PM']
        shots['3p_attempts'] += (teams_df[str(i)+'Q 3P'] + teams_df[str(i)+'Q 3PM'])
        #print(teams_df[str(i)+'Q FG'] + teams_df[str(i)+'Q FGM'] )
        #print(teams_df[str(i)+'Q FG'])
        #print(teams_df[str(i)+'Q FGM'])
        #print(shots['attempts'] )

    #true_shooting_attempts = (fg_attempts + three_attempts) + .44 * ft_attempts
    points = teams_df['points'].sum()
    print(shots['attempts'] )
    print(pd.concat([teams_df, shots], axis=1, ignore_index=False))
    print(points, fg_attempts, three_attempts, ft_attempts)
    ts_percentage = points / (2 * ( (fg_attempts + three_attempts) + .44 * ft_attempts))
    return ts_percentage

In [50]:
df0 = advanced_stats(teams_df=otf_df[otf_df['team'] == 'Nets'])
df0

0   NaN
Name: attempts, dtype: float64
78.0 28.0 32.0 14.0


0.5894800483675937

In [51]:
df1 = advanced_stats(teams_df=otf_df[otf_df['team'] == 'Heat'])
df1

0    32.0
Name: attempts, dtype: float64
66.0 32.0 22.0 6.0


0.5826271186440678

In [74]:
nets = grouped_score[grouped_score['team'] == 'Nets'].reset_index(drop=True)


In [61]:
#nets = nets.drop(['player', 'team', 'initials'], axis=1)


In [62]:
nets.sum()

player      Alize JohnsonBlake GriffinBruce BrownJeff Gree...
team                 NetsNetsNetsNetsNetsNetsNetsNetsNetsNets
points                                                  107.0
1Q PTS                                                   31.0
2Q PTS                                                   29.0
3Q PTS                                                   24.0
4Q PTS                                                   23.0
1Q FG                                                     5.0
2Q FG                                                     6.0
3Q FG                                                     7.0
4Q FG                                                     4.0
1Q FGM                                                    3.0
2Q FGM                                                    4.0
3Q FGM                                                    5.0
4Q FGM                                                    4.0
1Q 3P                                                     7.0
2Q 3P   

In [81]:
df2 = advanced_stats(teams_df=nets)

0     1.0
1     7.0
2     7.0
3     9.0
4    12.0
5     3.0
6    17.0
7    15.0
8     3.0
9     5.0
Name: attempts, dtype: float64
                    player  team  points  1Q PTS  2Q PTS  3Q PTS  4Q PTS  \
0            Alize Johnson  Nets     3.0     3.0     0.0     0.0     0.0   
1            Blake Griffin  Nets     7.0     3.0     1.0     3.0     0.0   
2              Bruce Brown  Nets     8.0     0.0     0.0     4.0     4.0   
3               Jeff Green  Nets     8.0     2.0     2.0     0.0     4.0   
4               Joe Harris  Nets    12.0     0.0     2.0     5.0     5.0   
5             Kevin Durant  Nets     8.0     8.0     0.0     0.0     0.0   
6             Kyrie Irving  Nets    20.0     2.0    12.0     4.0     2.0   
7            Landry Shamet  Nets    30.0     8.0    10.0     4.0     8.0   
8          Nicolas Claxton  Nets     4.0     2.0     0.0     2.0     0.0   
9  Timothe Luwawu-Cabarrot  Nets     7.0     3.0     2.0     2.0     0.0   

   1Q FG  2Q FG  3Q FG  ...  2Q 

In [43]:

heat = grouped_score[grouped_score['team'] == 'Heat']
heat

,player,team,points,1Q PTS,2Q PTS,3Q PTS,4Q PTS,1Q FG,2Q FG,3Q FG,...,4Q 3PM,1Q FT,2Q FT,3Q FT,4Q FT,1Q FTM,2Q FTM,3Q FTM,4Q FTM,initials
1,Andre Iguodala,Heat,8.0,0.0,5.0,3.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,A Iguodala
2,Bam Adebayo,Heat,21.0,2.0,8.0,2.0,9.0,1.0,3.0,1.0,...,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,B Adebayo
5,Dewayne Dedmon,Heat,10.0,2.0,6.0,2.0,0.0,0.0,1.0,0.0,...,0.0,2.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,D Dedmon
6,Duncan Robinson,Heat,11.0,3.0,3.0,2.0,3.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D Robinson
7,Goran Dragic,Heat,18.0,8.0,3.0,3.0,4.0,0.0,0.0,1.0,...,2.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,G Dragic
11,Kendrick Nunn,Heat,17.0,8.0,3.0,6.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,K Nunn
17,Trevor Ariza,Heat,15.0,9.0,2.0,2.0,2.0,0.0,0.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,T Ariza
18,Tyler Herro,Heat,9.0,0.0,2.0,2.0,5.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T Herro


In [44]:
heat = heat.drop(['player', 'team', 'initials'], axis=1)
heat.sum()

points    109.0
1Q PTS     32.0
2Q PTS     32.0
3Q PTS     22.0
4Q PTS     23.0
1Q FG       2.0
2Q FG       6.0
3Q FG       6.0
4Q FG       8.0
1Q FGM      5.0
2Q FGM      3.0
3Q FGM      8.0
4Q FGM      5.0
1Q 3P       8.0
2Q 3P       4.0
3Q 3P       2.0
4Q 3P       2.0
1Q 3PM      5.0
2Q 3PM      3.0
3Q 3PM      8.0
4Q 3PM      5.0
1Q FT       4.0
2Q FT       8.0
3Q FT       4.0
4Q FT       1.0
1Q FTM      0.0
2Q FTM      0.0
3Q FTM      0.0
4Q FTM      2.0
dtype: float64

In [45]:
df3 = advanced_stats(teams_df=heat.astype(int))
df3


109 43 37 19


0.6167949298325034